In [4]:
import pandas as pd

# Ruta del archivo Excel (ajústala según tu entorno)
file_path = "Productos validados.xlsx"

# Cargar la hoja correcta
df = pd.read_excel(file_path, sheet_name="Productos validados - INV")

# Extraer la columna de autores
columna_autores = "Autores | *Autor de correspondencia | ªEstudiante"

# Eliminar filas vacías en la columna relevante
df = df.dropna(subset=[columna_autores])

# Función para clasificar autores
def clasificar_autores(autores_str):
    investigadores = []
    estudiantes = []
    autores_correspondencia = []

    # Separar por ";"
    autores = autores_str.split(";")

    for autor in autores:
        autor = autor.strip()  # Eliminar espacios extras
        
        if autor.startswith("ª"):  # Si es estudiante
            estudiantes.append(autor[1:].strip())  # Remover "ª" y limpiar espacios
        else:
            autor_limpio = autor.lstrip("*").strip()  # Remover "*" si lo tiene y limpiar espacios
            investigadores.append(autor_limpio)
            if autor.startswith("*"):  # Si era autor de correspondencia
                autores_correspondencia.append(autor_limpio)

    return investigadores, estudiantes, autores_correspondencia

# Listas finales
lista_investigadores = []
lista_estudiantes = []
lista_autores_correspondencia = []

# Aplicar la función a cada fila y consolidar listas
for autores_str in df[columna_autores]:
    inv, est, corr = clasificar_autores(autores_str)
    lista_investigadores.extend(inv)
    lista_estudiantes.extend(est)
    lista_autores_correspondencia.extend(corr)

# Eliminar duplicados y ordenar las listas
lista_investigadores = sorted(set(lista_investigadores))
lista_estudiantes = sorted(set(lista_estudiantes))
lista_autores_correspondencia = sorted(set(lista_autores_correspondencia))

# Mostrar una vista previa
print("Investigadores:", lista_investigadores[:10])
print("Estudiantes:", lista_estudiantes[:10])
print("Autores de Correspondencia:", lista_autores_correspondencia[:10])


Investigadores: ['A PALESTINA R.', 'ANDRADE TORRES A.', 'ASCACIO VALDES J.A.', 'Abad Rosales S.M.', 'Abdo de la Parra M.I.', 'Abo Markeb A.', 'Abril Valdez E.M.', 'Acedo Félix E.', 'Acevedo Sandoval O.', 'Acosta Muñiz C.H.']
Estudiantes: ['ANTONIO G.', 'Acedo Espinoza E.', 'Acosta Slane D.', 'Acuña Gallardo A.', 'Acuña Juanz A.', 'Acuña Kaldman C.', 'Aguilar Delgado M.', 'Aguilar Rendón K.', 'Aguirre Sánchez J.', 'Agustín Chávez M.']
Autores de Correspondencia: ['ASCACIO VALDES J.A.', 'Abad Rosales S.M.', 'Abdo de la Parra M.I.', 'Abril Valdez E.M.', 'Acosta Muñiz C.H.', 'Acuña Kaldman C.', 'Aguilar C.N.', 'Aguilar Juárez M.', 'Aguilar Navarro C.', 'Aguilar Pérez V.H.']


In [6]:
import json
from itertools import combinations

# Ruta del archivo Excel (ajústala según tu entorno)
file_path = "Productos validados.xlsx"

# Cargar la hoja correcta
df = pd.read_excel(file_path, sheet_name="Productos validados - INV")

# Extraer la columna de autores
columna_autores = "Autores | *Autor de correspondencia | ªEstudiante"

# Eliminar filas vacías en la columna relevante
df = df.dropna(subset=[columna_autores])

# Diccionarios para asignar un ID único a cada investigador y estudiante
investigadores = set()
estudiantes = set()

# Extraer investigadores y estudiantes de cada publicación
for autores_str in df[columna_autores]:
    autores = autores_str.split(";")
    
    for autor in autores:
        autor = autor.strip()  # Quitar espacios en blanco
        
        if autor.startswith("ª"):  # Es estudiante
            estudiantes.add(autor[1:].strip())  # Quitar "ª" y limpiar
        else:  # Es investigador
            investigadores.add(autor.lstrip("*").strip())  # Quitar "*" y limpiar

# Convertir a listas ordenadas y asignar IDs
investigadores = sorted(investigadores)
estudiantes = sorted(estudiantes)
id_map = {name: f"inv_{i}" for i, name in enumerate(investigadores)}
id_map.update({name: f"est_{i}" for i, name in enumerate(estudiantes)})

# Crear nodos para cada investigador y estudiante
nodes = [{"data": {"id": id_map[name], "label": name, "type": "investigador"}} for name in investigadores]
nodes += [{"data": {"id": id_map[name], "label": name, "type": "estudiante"}} for name in estudiantes]

# Crear aristas (colaboraciones)
edges = []
for autores_str in df[columna_autores]:
    autores = [autor.strip() for autor in autores_str.split(";")]
    
    # Separar investigadores y estudiantes
    investigadores_pub = [a.lstrip("*").strip() for a in autores if not a.startswith("ª")]
    estudiantes_pub = [a[1:].strip() for a in autores if a.startswith("ª")]

    # Conectar investigadores entre sí
    for a, b in combinations(set(investigadores_pub), 2):
        edges.append({"data": {"source": id_map[a], "target": id_map[b], "label": "colaboración"}})

    # Conectar estudiantes con sus investigadores colaboradores
    for estudiante in estudiantes_pub:
        for investigador in investigadores_pub:
            edges.append({"data": {"source": id_map[estudiante], "target": id_map[investigador], "label": "asesoría"}})

# Construir el JSON final
graph_json = {
    "elements": {
        "nodes": nodes,
        "edges": edges
    }
}

# Guardar como archivo JSON
json_path = "grafo_cytoscape.json"
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(graph_json, f, indent=4, ensure_ascii=False)

print(f"Archivo JSON generado: {json_path}")


Archivo JSON generado: grafo_cytoscape.json


In [3]:
import pandas as pd
pd.read_excel('Productos validados.xlsx').to_csv('productos validados.csv')